In [1]:
import pandas as pd
from modules.data_preprocessor import DataPreprocessor
from modules.forecaster import Forecaster
from flask import Flask, jsonify
from modules.data_extractor import Extractor
from modules.evaluator import ForecastEvaluator
from modules.pipeline import *
from modules.client_config_manager import ConfigManager

from modules.utils.user_configurations import PASSWORDS
from modules.pipeline import run_pipeline, postprocess_predictions, get_latest_files, clean_future_data, clean_clients_predictions_df, get_ensemble_predictions, add_extra_covariates
import numpy as np

from modules.utils.metrics import (
    rmsse,
    bias,
    mae,
)
from modules.utils.forecasting import StatisticalForecaster, GlobalForecaster
from dotenv import load_dotenv
load_dotenv()
import os

/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# set password example
password = "m5_test_2_password"
client = PASSWORDS.get(password)

In [3]:
# initialize config manager
config_manager = ConfigManager(os.getenv("BUCKET_NAME"), client)
s3_config = config_manager.get_s3_config()
s3_config

{'bucket_name': 'tetra-forecasting',
 'base_path': 'm5_test_2',
 'data_path': 'm5_test_2/data',
 'plots_path': 'm5_test_2/plots',
 'configurations_path': 'm5_test_2/configurations',
 'model_path': 'm5_test_2/model'}

In [4]:
# Predict -> assuming hyperparams are tuned
model_config = config_manager.get_config("DEMO_MODEL_CONFIGURATIONS")
forecast_config = config_manager.get_config("DEMO_FORECAST_CONFIGURATIONS")
benchmark_config = config_manager.get_config("DEMO_BENCHMARK_CONFIGURATIONS")
#wednesdays_off_path  =benchmark_config["wednesdays_off_path"]
wednesdays_off_path = "/home/filtheo/tetra_product/modules/utils/configurations/extra_days.csv"

In [5]:
latest_file, latest_main_file, latest_forecast_file, latest_evaluation_file, latest_confirmed_file  = get_latest_files(config_manager)
print(latest_file, latest_main_file, latest_forecast_file, latest_evaluation_file, latest_confirmed_file)

20032016.csv main_13032016.csv forecasts_2016-03-20.csv evaluation_20032016.csv None


In [6]:
# Data extraction
extractor = Extractor(
        log_in_aws=True,
        bucket_path=s3_config['data_path'],
        bucket_name=s3_config['bucket_name'],
        original_file_name=latest_main_file,
        new_file_name=latest_file,
        client_name=config_manager.client_folder
    )

full_data, newest_data = extractor.extract_data(client_name=config_manager.client_folder)
future_data = extractor.extract_predicted_data()
forecasted_data = extractor.extract_latest_predictions(latest_forecast_file)

/home/filtheo/Cloud-for-AI-VIVES/modules/data_extractor.py:140: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(obj["Body"])


In [7]:
# Preprocess - Prepare
preprocessor = DataPreprocessor(full_data)
full_data = preprocessor.preprocess(remove_weekends=forecast_config["remove_weekends"],
                                        #item_time_covars=forecast_config["item_time_covars"],
                                        filter_dates=True,
                                        )

In [8]:
# Model predictions
forecaster = Forecaster(
    model_config=model_config,
    forecast_config=forecast_config,
    seasonal_length=forecast_config["seasonal_length"]
)
forecaster.fit(full_data)

# Get the predictions
predictions_c = forecaster.predict()

In [9]:
predictions = postprocess_predictions(predictions_c, ensemble = False)
predictions = concat_with_true_values(predictions, newest_data)
predictions

Dates are different


,unique_id,date,Model,y,fh,cv,Color,True
0,FOODS_1_001,2016-03-14,SeasonXpert,0.0,1,None,green,NaN
1,FOODS_1_001,2016-03-15,SeasonXpert,0.0,2,None,green,NaN
2,FOODS_1_001,2016-03-16,SeasonXpert,0.0,3,None,green,NaN
3,FOODS_1_001,2016-03-17,SeasonXpert,0.0,4,None,green,NaN
4,FOODS_1_001,2016-03-18,SeasonXpert,0.0,5,None,green,NaN
...,...,...,...,...,...,...,...,...
20113,FOODS_3_827,2016-03-16,SeasonalNaive,0.0,3,None,green,NaN
20114,FOODS_3_827,2016-03-17,SeasonalNaive,0.0,4,None,green,NaN
20115,FOODS_3_827,2016-03-18,SeasonalNaive,0.0,5,None,green,NaN
20116,FOODS_3_827,2016-03-19,SeasonalNaive,0.0,6,None,green,NaN


In [14]:
predictions_json = predict_jsonify(predictions)
predictions_json

{'FOODS_1_001': [{'date': '2016-03-14',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-15',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-16',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-17',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-18',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-19',
   'y': 0.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-20',
   'y': 1.0,
   'Model': 'SeasonXpert',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-14',
   'y': 0.0,
   'Model': 'SeasonalNaive',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-15',
   'y': 1.0,
   'Model': 'SeasonalNaive',
   'Color': 'green',
   'True': nan},
  {'date': '2016-03-16',
   'y': 0.0,
   'Model': 'Sea